In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/abalone/abalone_csv.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
data = pd.read_csv("/kaggle/input/abalone/abalone_csv.csv", sep=",", header='infer' )

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.utils import compute_class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [5]:
data.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Class_number_of_rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [6]:
category = np.repeat("empty000", data.shape[0])
category.size

4177

In [7]:
for i in range(0, data["Class_number_of_rings"].size):
    if(data["Class_number_of_rings"][i] <= 7):
        category[i] = int(1)
    elif(data["Class_number_of_rings"][i] > 7):
        category[i] = int(0)

In [8]:
category[10:25]

array(['0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0',
       '0', '0'], dtype='<U8')

In [9]:
data.Class_number_of_rings.value_counts()

9     689
10    634
8     568
11    487
7     391
12    267
6     259
13    203
14    126
5     115
15    103
16     67
17     58
4      57
18     42
19     32
20     26
3      15
21     14
23      9
22      6
24      2
27      2
1       1
25      1
2       1
26      1
29      1
Name: Class_number_of_rings, dtype: int64

In [10]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data['Sex'])
print(integer_encoded[0:7])

[2 2 0 2 1 1 0]


In [11]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
onehot_encoded[0:7]
#onehot_encoded.shape

array([[0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [12]:
data = data.drop(['Sex'], axis=1)
#print((onehot_encoded))
#size(np.array(onehot_encoded.tolist()))
#data['Gender'] = np.array(onehot_encoded.tolist())
data['category_size'] = category
data = data.drop(['Class_number_of_rings'], axis=1)

In [13]:
data.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,category_size
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,0
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,0
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,1


In [14]:
features = data.iloc[:,np.r_[0:7]]
labels = data.iloc[:,7]

In [15]:
data.category_size.value_counts()

0    3338
1     839
Name: category_size, dtype: int64

In [16]:
features.head()
#labels.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055


In [17]:
data.describe()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
count,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
mean,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831
std,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203
min,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500
25%,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000
50%,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000
75%,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000
max,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000


In [18]:
X_train, X_test, y_train, y_test, X_gender, X_gender_test = train_test_split(features, labels, onehot_encoded, random_state=10, test_size=0.2)

In [19]:
X_train.shape

(3341, 7)

In [20]:
X_gender.shape

(3341, 3)

In [21]:
y_train[0:3]

3544    0
2751    0
4019    0
Name: category_size, dtype: object

In [22]:
X_test.shape

(836, 7)

In [23]:
X_test[1:3]

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
1371,0.615,0.50,0.17,1.0540,0.4845,0.2280,0.2950
2591,0.585,0.45,0.16,0.9045,0.4050,0.2215,0.2335


In [24]:
temp = X_train.values
X_train_gender = np.concatenate((temp, X_gender), axis=1)
X_train_gender.shape

(3341, 10)

In [25]:
temp = X_test.values
X_test_gender = np.concatenate((temp, X_gender_test), axis=1)
X_test_gender.shape

(836, 10)

In [28]:
X_train_gender

array([[0.475, 0.355, 0.125, ..., 0.   , 1.   , 0.   ],
       [0.515, 0.395, 0.135, ..., 1.   , 0.   , 0.   ],
       [0.66 , 0.515, 0.155, ..., 0.   , 0.   , 1.   ],
       ...,
       [0.465, 0.355, 0.115, ..., 1.   , 0.   , 0.   ],
       [0.585, 0.47 , 0.165, ..., 0.   , 0.   , 1.   ],
       [0.505, 0.395, 0.105, ..., 0.   , 1.   , 0.   ]])

In [29]:
X_test_gender

array([[0.43 , 0.335, 0.12 , ..., 0.   , 0.   , 1.   ],
       [0.615, 0.5  , 0.17 , ..., 0.   , 0.   , 1.   ],
       [0.585, 0.45 , 0.16 , ..., 1.   , 0.   , 0.   ],
       ...,
       [0.215, 0.155, 0.06 , ..., 0.   , 1.   , 0.   ],
       [0.475, 0.375, 0.12 , ..., 0.   , 0.   , 1.   ],
       [0.49 , 0.36 , 0.11 , ..., 1.   , 0.   , 0.   ]])

In [30]:
from imblearn.over_sampling import SMOTE

In [31]:
y_train

3544    0
2751    0
4019    0
997     0
2523    0
       ..
2009    0
1180    0
3441    1
1344    0
1289    0
Name: category_size, Length: 3341, dtype: object

In [32]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(data['category_size']), y=data['category_size'])
class_weights_dict = dict(enumerate(class_weights))
class_weights_dict

{0: 0.6256740563211504, 1: 2.4892729439809296}

In [33]:
old_keys = np.r_[0:27]
new_keys = np.unique(data['category_size'])
weights_final = dict(zip(new_keys, list(class_weights_dict.values()))) 
#class_weights_dict[new_keys] = class_weights_dict.pop(old_keys)
#class_weights_dict.keys()
weights_final

{'0': 0.6256740563211504, '1': 2.4892729439809296}

In [34]:
X_train_gender[0:5]

array([[0.475 , 0.355 , 0.125 , 0.4865, 0.2155, 0.1105, 0.142 , 0.    ,
        1.    , 0.    ],
       [0.515 , 0.395 , 0.135 , 0.516 , 0.2015, 0.132 , 0.162 , 1.    ,
        0.    , 0.    ],
       [0.66  , 0.515 , 0.155 , 1.4415, 0.7055, 0.3555, 0.335 , 0.    ,
        0.    , 1.    ],
       [0.59  , 0.47  , 0.16  , 1.206 , 0.479 , 0.2425, 0.309 , 0.    ,
        0.    , 1.    ],
       [0.565 , 0.44  , 0.15  , 0.983 , 0.4475, 0.2355, 0.2485, 1.    ,
        0.    , 0.    ]])

In [35]:
y_train.value_counts()

0    2666
1     675
Name: category_size, dtype: int64

In [36]:
X_resampled, y_resampled = SMOTE().fit_sample(X_train_gender, y_train)

In [37]:
X_resampled.shape

(5332, 10)

In [38]:
X_train_gender.shape

(3341, 10)

In [39]:
y_resampled.value_counts()

1    2666
0    2666
Name: category_size, dtype: int64

In [40]:
y_resampled

0       0
1       0
2       0
3       0
4       0
       ..
5327    1
5328    1
5329    1
5330    1
5331    1
Name: category_size, Length: 5332, dtype: object

In [41]:
rf_classifier = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                                       max_features='auto', n_estimators=30, oob_score=False,n_jobs=-1)
rf_classifier.fit(X_train_gender, y_train)

RandomForestClassifier(n_estimators=30, n_jobs=-1)

In [42]:
rf_classifier_train = rf_classifier.predict(X_train_gender)
accuracy_score(y_train, rf_classifier_train)

0.9979048189164921

In [43]:
rf_classifier_test = rf_classifier.predict(X_test_gender)
accuracy_score(y_test, rf_classifier_test)

0.9007177033492823

In [44]:
rf_classifiert = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                                       max_features='auto', n_estimators=30, oob_score=False,n_jobs=-1)
rf_classifiert.fit(X_resampled, y_resampled)

RandomForestClassifier(n_estimators=30, n_jobs=-1)

In [45]:
rf_classifier_tran = rf_classifiert.predict(X_resampled)
accuracy_score(y_resampled, rf_classifier_tran)

0.9994373593398349

In [46]:
rf_classifier_tes = rf_classifiert.predict(X_test_gender)
accuracy_score(y_test, rf_classifier_tes)

0.881578947368421

In [47]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

In [48]:
Xn,yn=shuffle_in_unison(X_resampled, y_resampled)

In [54]:
rf_classifiert = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                                       max_features='auto', n_estimators=30, oob_score=False,n_jobs=-1)
rf_classifiert.fit(Xn, yn)

RandomForestClassifier(n_estimators=30, n_jobs=-1)

In [55]:
rf_classifier_tran = rf_classifiert.predict(Xn)
accuracy_score(yn, rf_classifier_tran)

0.9998124531132783

In [56]:
rf_classifier_tes = rf_classifiert.predict(X_test_gender)
accuracy_score(y_test, rf_classifier_tes)

0.8851674641148325

In [57]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train_gender, y_train)

SVC(kernel='linear')

In [58]:
y_pred = svclassifier.predict(X_test_gender)
accuracy_score(y_test, y_pred)

0.9055023923444976

In [59]:
svclassifier2 = SVC(kernel='linear')
svclassifier2.fit(Xn, yn)

SVC(kernel='linear')

In [60]:
y_pred2 = svclassifier2.predict(X_test_gender)
accuracy_score(y_test, y_pred2)

0.8624401913875598

In [61]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,rf_classifier_test))
print(classification_report(y_test,rf_classifier_test))

[[633  39]
 [ 44 120]]
              precision    recall  f1-score   support

           0       0.94      0.94      0.94       672
           1       0.75      0.73      0.74       164

    accuracy                           0.90       836
   macro avg       0.84      0.84      0.84       836
weighted avg       0.90      0.90      0.90       836



In [62]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,rf_classifier_tes))
print(classification_report(y_test,rf_classifier_tes))

[[606  66]
 [ 30 134]]
              precision    recall  f1-score   support

           0       0.95      0.90      0.93       672
           1       0.67      0.82      0.74       164

    accuracy                           0.89       836
   macro avg       0.81      0.86      0.83       836
weighted avg       0.90      0.89      0.89       836



In [63]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.3.1


In [74]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [75]:
model.compile(optimizer='adam',
              loss='BinaryCrossentropy',
              metrics=['accuracy'])

In [68]:
for i in range(5332):
    if(yn[i] == "1"):
        yn[i]=1
    elif(yn[i]=="0"):
        yn[i] = 0

In [70]:
yn.shape

(5332,)

In [71]:
y_test

1080    1
1371    0
2591    0
235     0
2143    0
       ..
1058    1
1323    0
784     1
2291    0
725     0
Name: category_size, Length: 836, dtype: object

In [81]:
Xn

array([[0.28      , 0.21854512, 0.07709023, ..., 0.        , 1.        ,
        0.        ],
       [0.505     , 0.4       , 0.13      , ..., 0.        , 0.        ,
        1.        ],
       [0.495     , 0.38      , 0.12      , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.49      , 0.38      , 0.145     , ..., 1.        , 0.        ,
        0.        ],
       [0.445     , 0.335     , 0.14      , ..., 1.        , 0.        ,
        0.        ],
       [0.275     , 0.2       , 0.07      , ..., 0.        , 1.        ,
        0.        ]])

In [93]:
l=[]
for i in range(len(yn)):
    l.append(1*yn[i])
l    

[1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,


In [97]:
ynn=np.array(l)
ynn.shape

(5332,)

In [98]:
model.fit(Xn,ynn, epochs=300)

Epoch 1/300
167/167 [==============================] - 0s 2ms/step - loss: 0.4677 - accuracy: 0.8181
Epoch 2/300
167/167 [==============================] - 0s 2ms/step - loss: 0.4222 - accuracy: 0.8462
Epoch 3/300
167/167 [==============================] - 0s 1ms/step - loss: 0.3654 - accuracy: 0.8607
Epoch 4/300
167/167 [==============================] - 0s 1ms/step - loss: 0.9153 - accuracy: 0.7894
Epoch 5/300
167/167 [==============================] - 0s 1ms/step - loss: 0.4227 - accuracy: 0.8593
Epoch 6/300
167/167 [==============================] - 0s 2ms/step - loss: 0.3909 - accuracy: 0.8573
Epoch 7/300
167/167 [==============================] - 0s 2ms/step - loss: 0.3647 - accuracy: 0.8659
Epoch 8/300
167/167 [==============================] - 0s 2ms/step - loss: 0.4790 - accuracy: 0.8239
Epoch 9/300
167/167 [==============================] - 0s 2ms/step - loss: 1.5224 - accuracy: 0.7494
Epoch 10/300
167/167 [==============================] - 0s 2ms/step - loss: 0.5950 - accura

In [100]:
type(y_test)

pandas.core.series.Series

In [101]:
yt=y_test.to_numpy()

In [102]:
yt

array(['1', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0',
       '0', '1', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0',
       '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0',
       '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '0', '0',
       '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '1', '1', '0', '0', '0', '0', '0', '0',
       '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '1', '1',
       '0', '0', '0', '1', '0', '1', '0', '0', '0', '0', '0', '0

In [107]:
int(yt[0])

1

In [108]:
l2=[]
for j in range(len(yt)):
    l2.append(1*int(yt[j]))
l2    

[1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,


In [109]:
ytn=np.array(l2)
ytn.shape

(836,)

In [110]:
X_test_gender.shape

(836, 10)

In [111]:
Xtest_loss, test_acc = model.evaluate(X_test_gender, ytn , verbose=2)

print('\nTest accuracy:', test_acc)


27/27 - 0s - loss: 0.3652 - accuracy: 0.8612

Test accuracy: 0.8612440228462219


In [112]:
ypre=model.predict(X_test_gender)

In [113]:
ypre= np.ravel(ypre)
ypre.shape

(836,)

In [131]:
ypr=(ypre>0.565)*1
ypr


array([1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [132]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(ytn,ypr))
print(classification_report(ytn,ypr))

[[600  72]
 [ 24 140]]
              precision    recall  f1-score   support

           0       0.96      0.89      0.93       672
           1       0.66      0.85      0.74       164

    accuracy                           0.89       836
   macro avg       0.81      0.87      0.84       836
weighted avg       0.90      0.89      0.89       836



In [134]:
ytn.shape

(836,)

In [ ]:
ypr.shape